# Demonstration of some Pandas capabilities using a famous concrete dataset

First, we are going to import some necessary libraries and to load the famous concrete dataset published by Yeh et al.
To save typing, we have created a tinyurl `tunyurl.com/icccm1` which contains a link to a `xls` file containing the dataset.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tgaedt/workshop_uv/blob/main/notebooks/1_pandas_demo.ipynb)



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

url = "https://tinyurl.com/icccm1"
df = pd.read_excel(url) 

df.head(3)

We note that the column names are very long and contain some special characers. Therefore, we will manually rename the columns.

In [ ]:
df.columns = ["cement", "bfs", "flyash", "water", "sp", "agg_coarse", "agg_fine", "age", "strength"]
df.head(3)

In [ ]:
# get the length of the dataframe
df_length = len(df)
df_length

## Different ways to select a column in Pandas
We now demonstrate how to select a column with Pandas. Note that the type of a column is a Pandas Series.

In [ ]:
type(df.cement)

A specific column, here the column containing the amount of cement can be selected using a dot notation like this:

In [ ]:
df.cement.head(4)

Alternatively a notation with square brackets is also possible:

In [ ]:
df["cement"].head(4)

It is also possible to use in indexed location based access. The `:` means that all rows are selected.

In [ ]:
df.iloc[:,0].head(4)

A name based selection is also possible

In [ ]:
col4 = df.loc[:,"cement"]

## Create a new column - calculate the w/c value

A new column can be created with a simple assignment like `df["new_col_name"]`. 
Alternatively the methods `assign()` or `insert()` can be used.

In [ ]:
df["wc"] = df["water"] / (df["cement"] + df["bfs"] + df["flyash"])
df.head(4)

A column can be deleted with the method `drop()`, using the parameter `inplace = True` enables to leave out the assignment `df = df.drop()`.

In [ ]:
df.drop(columns=["wc"], inplace=True)
df.head(3)

Next, we show the assign and insert methods.

In [ ]:
df = df.assign(wc = df["water"] / (df["cement"] + df["bfs"] + df["flyash"]))
df.head(3)

In [ ]:
# delete the column again
df.drop(columns=["wc"], inplace=True)

# the insert method allows to specify the column index
df.insert(4, "wc", df["water"] / (df["cement"] + df["bfs"] + df["flyash"]))
df.iloc[2:5]

It is probably a good idea to round the water-to-cement ratio after two decimal positions.

In [ ]:
# better to round off after the second decimal
df["wc"] = df["wc"].round(2)
df.iloc[2:5]

## Data types
Currently, the dataframe only has float and int as datatypes.

In [ ]:
df.dtypes

We are now going to create a new column which contains entries of type `boolean`. Therefore, we are going to chose an arbitrary criterium to check. Here, we are going to evaluate whether the w/c value is higher than 0.42. We will name the column `wc_high`. 

In [ ]:
df.insert(5, "wc_high", df["wc"] > 0.42)
df.head(4)

In [ ]:
# check dtypes
df.dtypes

## Calculating overview metrics of the dataset
Because `True` will evaluate to 1, we can easily count the number of concretes with a w/c ratio of larger than 0.42. 

In [ ]:
# count high wc instances
nr_high = df.wc_high.sum()

nr_high

Of course, this can be also summarized in one line for quick counting. No need to create a new column. Let's count the concretes with more than 100 kg of slag in the mix design.

In [ ]:
(df["bfs"] > 100).sum()

It is also easy and straightforward to obtrain the mean values of all columns.

In [ ]:
df.mean(numeric_only=True)

Pandas has a nice method (`desribe()`) for getting a quick overview about the data properties in the dataframe. 

In [ ]:
df.describe()

## Filtering a dataset
Very often, you need to filter a dataframe by one or more criteria. This can be achieved in differnt ways. 

In [ ]:
# filter operations
df_high = df.query("wc_high > 0.42")
df_high.head(3)


One thing we has irritated me many, many times is the concept of a **view** in Pandas. The `df.query("wc_high > 0.42")` method does not necessarily creates a filtered copy of the original dataframe, i.e. a **new**, independent dataframe to the variable `df_high`. You can get a `SettingWithCopyWarning`. For details see https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas

In [ ]:
df_high["new_var"] = 5 

The warning can be avoided by calling the `copy()` method.

In [ ]:
df_high = df.query("wc_high > 0.42").copy()
df_high["new_var"] = 5

Another method to filter dataframes is to create a filter mask.

In [ ]:
mask = (df["wc"] > 0.48) & (df["bfs"] < 50)
mask.sum()

Now the mask can be applied to a dataframe to create a filtered dataframe.

In [ ]:
df_filtered = df[mask]

## Joining datasets
Often, two datasets need to be joined. To do a join operation, a common key is necessary for both dataframes. We can not go into details here but would like to illustrate this point using simple, artifical example. 

First, we need to create a key in the existing concrete dataset. In our case, suitable keys are often the names or codes of the individual experiment. Here, we will create a new column with strings `concrete_mix_1` etc. To create this list, we will use the index values of the original dataframe as numbers. 
Note that every dataframe object in pandas has an index. In the current case the index is just a list of numbers called `RangeIndex`.

In [ ]:
df.index

We can loop through the index and get the inidividual numbers.

In [ ]:
for number in df.index[:6]:
    print(number)

A more concise way to do this in Python is the use of a list comprehension.

In [ ]:
[number for number in df.index[:6]]

We can now combine a list comprehension which loops through all values in the index with a string **concrete_mix_**. Note that we need to transform the number from the index into a string object, i.e., we need to use `str(i)`.

In [ ]:
exp_names = ["concrete_mix_" + str(i) for i in df.index.values]
df["exp_code"] = exp_names
df.head(3)

We will now create a second dataframe which also contains a column with the strings **concrete_mix_** and some random values in a second column.

In [ ]:
additional_data = np.random.randint(0, 300, len(df)) 
df2 = pd.DataFrame({"exp_code":exp_names, "new_parameter": additional_data})

df2.head(3)

Now the two dataframes `df` and `df2` can easily be joined.

In [ ]:
# join by index
df_joined = pd.merge(df, df2, on="exp_code")

df_joined.head(3)

## Visualization and exploratory data analysis
The exploratory data analysis of a dataset is very important in understanding the relations between the parameters (i.e., the columns). For the concrete dataset we have 1030 compressive strength tests. We already know that the compressive strength of a concrete depends on the w/c value, the curing duration, and the cement content. 

We begin by exploring the relation between w/c ratio and strength.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df["wc"], df["strength"])
ax.set_xlabel("w/c")
ax.set_ylabel("Strength / MPa")
plt.show()

There is a clear downward trend in the data with increasing w/c values. The most important parameter to take into account now is the curing age of the concrete. We will filter the dataset to include 3, 7, and 28 days.

In [ ]:
for name, group in df.query("age in [3, 7, 28]").groupby("age"):
    fig, ax = plt.subplots()
    ax.scatter(group["wc"], group["strength"])
    ax.set_title(f"Age {name} days")
    ax.set_xlabel("w/c")
    ax.set_ylabel("Strength / MPa")
    plt.show()
    


## Subplots
It would be nice to create a diagram with subplots so that the differences are far more easy to distinguish. This can be done in matplotlib using subplots. Here we are creating 1 row and 3 columns of subplots.

Next we need to loop through the filtered and grouped data and simultaneously through the list containing the axes objects for the individual subplots. This can be achieved with the `zip` command in python. Because the `groupby` method returns two objects, we need to use brackets with the declaration of the loop. The parameter `observed = True` is necessary to suppress a deprecation warning. Feel free to ignore this at the moment.

In [ ]:
fig, axs = plt.subplots(1,3, layout="tight", figsize=(9,4))

for (name, group), ax in zip(df.query("age in [3, 7, 28]").groupby("age", observed=True), axs.flatten()):
    ax.scatter(group["wc"], group["strength"], s=4)
    ax.set_title(f" Age {name} days")
    ax.set_xlabel("w/c")
    ax.set_ylim(0,90)
axs[0].set_ylabel("Strength / MPa")
plt.show()


## Fast and simple correlation matrix using seaborn

First, we will show how to get all correlation coefficents between the columns quickly.

In [ ]:
correlation_matrix = df.select_dtypes(include=["number"]).corr()

Next, we will leverage a plotting library which builds on matplotlib and makes some plotting objectives much easier to achieve.

In [ ]:

import seaborn as sns
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.show()

## Fitting the data (grouped)

One way to illustrate the true usefulness of Python is to demonstrate how easy it is to fit the data in a grouped fashion. We are going to use the method `curve_fit` from scipy to do this.

First, we define an exponential function `exponential`

$$
\sigma_c = c + a \cdot \exp(-b\cdot w/c)
$$

As we want to group our dataset by the age and then apply a fit function, we define `fit_exponential` which returns both the fit parameters and the R2 value for the quality of the fit. 

In [ ]:

from scipy.optimize import curve_fit

# define function to fit
def exponential(x, a, b):
    return a * np.exp(-b*x)

# define function which returns the best fit parameters as a pandas Series
def fit_exponential(df, xcol="wc", ycol="strength"):
    f = exponential
    popt, pcov = curve_fit(f, df[xcol], df[ycol], maxfev=10000)
    
    # Calculate R-squared
    y_actual = df[ycol]
    y_predicted = f(df[xcol], *popt)
    ss_res = np.sum((y_actual - y_predicted) ** 2)
    ss_tot = np.sum((y_actual - np.mean(y_actual)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    
    out = {"a": popt[0], "b": popt[1], "r_squared": r_squared}
    return pd.Series(out)

fits = df.query("age in [3, 7, 28]").groupby("age").apply(lambda t: fit_exponential(t), include_groups=False)

fits

## Plot the fitted curves and the raw data
It is very important to check the quality of the fits. We are going to plot the fits together with the fitted raw data. 

In [ ]:
x = np.linspace(df.wc.min(), df.wc.max(),100)

fig, ax = plt.subplots()
for index, row in fits.iterrows():
    y = exponential(x, row["a"], row["b"])
    ax.plot(x,y)
for name, group in df.query("age in [3,7,28] ").groupby("age"):
    ax.scatter(group["wc"], group["strength"], s=8, alpha =0.5, label=name)
ax.set_xlim(0.22, 0.8)
ax.set_ylim(0, 90)
ax.set_xlabel("w/c")
ax.set_ylabel("$\sigma_c$ / MPa")
plt.legend(title="Age / Days")
plt.plot()